In [643]:
import pandas as pd
import sqlite3 as sql
import numpy as np

In [431]:
## read in excel sheets
zip_to_grid = pd.read_excel('/Users/josheverts/Downloads/Zip_to_Grid.xlsx')
epa_df = pd.read_xml("/Users/josheverts/Downloads/vehicles.xml")
epa_test_data = pd.read_csv('/Users/josheverts/Downloads/light-duty-vehicle-test-results-report-2014-present.csv', header = 1)
test_data_2023 = pd.read_excel('/Users/josheverts/Downloads/23-testcar-2022-11-03.xlsx')


<ipython-input-431-a6df553caf28>:4: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  epa_test_data = pd.read_csv('/Users/josheverts/Downloads/light-duty-vehicle-test-results-report-2014-present.csv', header = 1)


In [703]:
xls = pd.ExcelFile('/Users/josheverts/Downloads/GREET_2022/GREET2_2022.xlsm')
df1 = pd.read_excel(xls, 'Mat_Sum')
df2 = pd.read_excel(xls, 'Battery_Sum')
df3 = pd.read_excel(xls, 'Vehi_Comp_Sum')

/Users/josheverts/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/josheverts/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/Users/josheverts/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [704]:
## clean material sum file and extract materials;
## assign each material an adjusted row index 
df1_subset = df1.iloc[24:]
column_names = df1_subset.iloc[0]
column_names[0] = 'Emissions Type'
df1_subset.columns = column_names
df1_emissions_subset = df1_subset.iloc[32-24:46-24] ## adjust to new index
df1_emissions_subset = df1_emissions_subset.drop([32])


In [714]:
## clean battery percentages file
df2_subset_NiMh = df2.iloc[48:90]
columns = df2_subset.iloc[0]
columns[0] = 'Material'
## create two separate tables for NiMh and Li-ion batteries
## NiMh battery
df2_subset_NIMH = df2_subset.iloc[57-48:67-48]
df2_subset_NIMH.columns = columns
df2_subset_NIMH = df2_subset_NIMH.drop(df2_subset_NIMH.columns[[1,2]], axis = 1)
df2_subset_NIMH = df2_subset_NIMH.drop(df2_subset_NIMH.columns[9:], axis = 1)
## LiIon battery
df2_subset_LI = df2_subset.iloc[68-48:88-48]
df2_subset_LI.columns = columns
df2_subset_LI = df2_subset_LI.drop(df2_subset_LI.columns[[1,2]], axis = 1)
df2_subset_LI = df2_subset_LI.drop(df2_subset_LI.columns[9:], axis = 1)


In [715]:
df2_subset_LI

48,Material,HEV: Conventional Material,HEV: Lightweight Material,PHEV: Conventional Material,PHEV: Lightweight Material,EV: Conventional Material,EV: Lightweight Material,FCV: Conventional Material,FCV: Lightweight Material
68,Active Material,0.1029,0.1029,0.156,0.156,0.2504,0.2504,0.1079,0.1079
69,Graphite/Carbon,0.0627,0.0627,0.0915,0.0915,0.1421,0.1421,0.0655,0.0655
70,Silicon,0,0,0,0,0,0,0,0
71,Binder,0.0034,0.0034,0.005,0.005,0.008,0.008,0.0035,0.0035
72,Copper,0.1386,0.1386,0.0867,0.0867,0.0744,0.0744,0.1425,0.1425
73,Wrought Aluminum,0.1718,0.1718,0.1439,0.1439,0.1214,0.1214,0.1709,0.1709
74,Cast Aluminum,0,0,0,0,0,0,0,0
75,Electrolyte: LiPF6,0.007,0.007,0.0059,0.0059,0.0093,0.0093,0.0073,0.0073
76,Electrolyte: Ethylene Carbonate,0.0196,0.0196,0.0166,0.0166,0.0261,0.0261,0.0205,0.0205
77,Electrolyte: Dimethyl Carbonate,0.0196,0.0196,0.0166,0.0166,0.0261,0.0261,0.0205,0.0205


In [716]:
## clean vehicle composition percentages file
df3_subset = df3.iloc[134:174]
columns = df2_subset.iloc[0]
columns[0] = 'Material'
df3_subset.columns = columns
df3_subset = df3_subset.drop(df3_subset.columns[11:], axis = 1)

In [717]:
## process materials df for later joins
df1_emissions_subset_t = df1_emissions_subset.T
columns = df1_emissions_subset_t.iloc[0]
df1_emissions_subset_t.index.name = None
df1_emissions_subset_t.columns = columns
df1_emisisons_subset_t = df1_emissions_subset_t.insert(0, 'Material', df1_emissions_subset_t.index)
df1_emissions_subset_t.reset_index(drop = True, inplace = True)
df1_emissions_subset_t.rename_axis(None, axis=1)
df1_emissions_subset_t = df1_emissions_subset_t.drop([0])

In [718]:
def get_alphabetical_index(ind):
    ## index starts at 1 in df, which matches to 'b' 
    ## this behavior matches GREET table where materials start at 'b'
    itr = ind//26
    rem = ind%26
    if itr > 0:
        return string.ascii_lowercase[itr-1] + string.ascii_lowercase[rem]
    return string.ascii_lowercase[rem]
    
import re
def get_mat_codes(excel_func):
    ## looks for "XY12" and returns XY 
    codes = []
    split_substr = excel_func.split('Mat_Sum!$')
    for val in split_substr[1:]: ## search after first Mat_Sum onwards
        search = re.search('[a-zA-Z]{1,2}',val)
        if search:
            codes.append(search[0])
    return codes

def get_LI_battery_matches(excel_func, *args):
    ## look for material index in function string
    ## assume that table is called Mat_Sum (as it is in GREET)
    ## split string on lookup for battery chemistry type (specified with *args)
    ## then use get_mat_codes on result strings
    if args:
        split_str = excel_func.split('$C$104='+str(args[0]))
        ## look after split
        ## get first match
        matches = re.findall(r"(?:)Mat_Sum(?:$|\W{2})(?:)\D{1,3}", split_str[1])
        match_1 = matches[0]
        all_matches = []
        all_matches.append(match_1)
        ## split after on "+" symbol which signifies end of conditional string region
        split_after = split_str[1].split('+') ## get all substrings after first "+" (index 1:)
        for val in split_after[1:]:
#             print(val)
            matches = re.findall(r"(?:)Mat_Sum(?:$|\W{2})(?:)\D{1,3}", val)
            if matches:
                for match in matches:
                    all_matches.append(match)
#                     matches.append(match)
        ## get cell index for each code
        mat_codes = []
        for match in all_matches:
            mat_codes.append(get_mat_codes(match))
        return mat_codes
    return None


In [719]:
alphabetical_ind = [get_alphabetical_index(i) for i in df1_emissions_subset_t.index]

In [720]:
df1_emisons_subset_t = df1_emissions_subset_t.insert(0, 'alph_index', alphabetical_ind)

In [721]:
excel_func_bat_comp_LI = '=((I$179*(IF(Vehi_Inputs!$C$104=1,Mat_Sum!$BG45,IF(Vehi_Inputs!$C$104=2,Mat_Sum!$BI45,IF(Vehi_Inputs!$C$104=3,Mat_Sum!$BO45,IF(Vehi_Inputs!$C$104=4,Mat_Sum!$BP45,IF(Vehi_Inputs!$C$104=5,Mat_Sum!$BK45,IF(Vehi_Inputs!$C$104=6,Mat_Sum!$BL45,IF(Vehi_Inputs!$C$104=7,Mat_Sum!$BS45,IF(Vehi_Inputs!$C$104=8,Mat_Sum!$BJ45,IF(Vehi_Inputs!$C$104=9,0)))))))))))+(I$179*Mat_Sum!$BT45*LiMn2O4!$E$9)+I$180*Mat_Sum!$BU45+I$181*Mat_Sum!$CC45+I$182*Mat_Sum!$BV45+I$183*Mat_Sum!$AC45+I$184*Mat_Sum!$J45+I$185*Mat_Sum!$N45+I$186*Mat_Sum!$BW45+I$187*Mat_Sum!$BX45+I$188*Mat_Sum!$BY45+I$189*Mat_Sum!$AM45+I$190*Mat_Sum!$AK45+I$191*Mat_Sum!$AL45+I$192*Mat_Sum!$D45+I$193*Mat_Sum!$BA45+I$194*Mat_Sum!$CA45+I$195*Mat_Sum!$BZ45+I$196*Mat_Sum!$AN45+I$197*Mat_Sum!$F45+I$198*Mat_Sum!$CH45+Battery_Assembly!$F78*$C$23)*(1+$D$30)'

In [722]:
excel_func_vehi_comp = '=B$192*Mat_Sum!$D45+B$193*Mat_Sum!$F45+B$194*Mat_Sum!$G45+B$195*Mat_Sum!$J45+B$196*Mat_Sum!$N45+B$197*Mat_Sum!$AC45+B$198*Mat_Sum!$AD45+B$199*Mat_Sum!$AG45+B$200*Mat_Sum!$AI45+B$201*Mat_Sum!$AN45+B$205*Mat_Sum!$AO45+B$202*Mat_Sum!$AR45+B$203*Mat_Sum!$AP45+B$204*Mat_Sum!$AQ45+B$206*Mat_Sum!$U45+B$207*Mat_Sum!$BD45+B$208*Mat_Sum!$BC45+B$209*Mat_Sum!$BF45+B$210*Mat_Sum!$BE45+B$211*Mat_Sum!$AS45+B$212*Mat_Sum!$CB45+B$213*Mat_Sum!$BU45+B$214*Mat_Sum!$CG45+B$215*0+B$216*0+B$217*Mat_Sum!$AL45+B$218*Mat_Sum!$AL45+B$219*0+B$220*Mat_Sum!$AJ45+B$221*0+B$222*Mat_Sum!$CF45+B$223*Mat_Sum!$CE45+B$224*0+B$225*Mat_Sum!$CL45+B$226*Mat_Sum!$CH45+B$227*Mat_Sum!$CI45+B$228*Mat_Sum!$AM45+B$229*Mat_Sum!$CK45+B$230*Mat_Sum!$CJ45+B$231*0'
excel_func_bat_comp_LI = '=((I$179*(IF(Vehi_Inputs!$C$104=1,Mat_Sum!$BG45,IF(Vehi_Inputs!$C$104=2,Mat_Sum!$BI45,IF(Vehi_Inputs!$C$104=3,Mat_Sum!$BO45,IF(Vehi_Inputs!$C$104=4,Mat_Sum!$BP45,IF(Vehi_Inputs!$C$104=5,Mat_Sum!$BK45,IF(Vehi_Inputs!$C$104=6,Mat_Sum!$BL45,IF(Vehi_Inputs!$C$104=7,Mat_Sum!$BS45,IF(Vehi_Inputs!$C$104=8,Mat_Sum!$BJ45,IF(Vehi_Inputs!$C$104=9,0)))))))))))+(I$179*Mat_Sum!$BT45*LiMn2O4!$E$9)+I$180*Mat_Sum!$BU45+I$181*Mat_Sum!$CC45+I$182*Mat_Sum!$BV45+I$183*Mat_Sum!$AC45+I$184*Mat_Sum!$J45+I$185*Mat_Sum!$N45+I$186*Mat_Sum!$BW45+I$187*Mat_Sum!$BX45+I$188*Mat_Sum!$BY45+I$189*Mat_Sum!$AM45+I$190*Mat_Sum!$AK45+I$191*Mat_Sum!$AL45+I$192*Mat_Sum!$D45+I$193*Mat_Sum!$BA45+I$194*Mat_Sum!$CA45+I$195*Mat_Sum!$BZ45+I$196*Mat_Sum!$AN45+I$197*Mat_Sum!$F45+I$198*Mat_Sum!$CH45+Battery_Assembly!$F78*$C$23)*(1+$D$30)'
excel_func_bat_comp_NIMH = '=(I$135*Mat_Sum!$G46+I$136*Mat_Sum!$D46+I$137*Mat_Sum!$J46+I$138*Mat_Sum!$AC46+I$139*Mat_Sum!$AG46+I$140*Mat_Sum!$AB46+I$141*Mat_Sum!$U46+I$142*Mat_Sum!$AX46+I$143*Mat_Sum!$AN46+I$144*Mat_Sum!$AR46+(I$46*Battery_Assembly!$E79))*(1+$C$30)'
codes_df2_LI = get_LI_battery_matches(excel_func_bat_comp_LI, 2)
codes_df2_LI = list(np.array(codes_df2_LI).squeeze())
codes_df2_LI = [codes_df2_LI[0]] +  codes_df2_LI[2:] ## can come back to this later, it is messing up join and unclear what this term does
codes_df2_NIMH = get_mat_codes(excel_func_bat_comp_NIMH)
codes_df3 = get_mat_codes(excel_func_vehi_comp)

## lowercase all values for comparison with main dataframe
codes_df2_LI = [i.lower() for i in codes_df2_LI]
codes_df2_NIMH = [i.lower() for i in codes_df2_NIMH]
codes_df3 = [i.lower() for i in codes_df3]

## place codes into respective dataframes
df2_subset_LI.insert(0, 'alph_index', codes_df2_LI)
df2_subset_NIMH.insert(0, 'alph_index', codes_df2_NIMH)
## clean df3_subset to drop unused materials first
unused_material_inds = [215, 216, 219, 221, 224, 231]
unused_material_inds = [i-193 for i in unused_material_inds]
unused_material_inds = [i+134 for i in unused_material_inds]
df3_subset = df3_subset[~df3_subset.index.isin(unused_material_inds)]
df3_subset.insert(0, 'alph_index', codes_df3)

In [723]:
## get materials density dataframes for each index set
emissions_factor_df2_LI = df1_emissions_subset_t[df1_emissions_subset_t['alph_index'].isin(codes_df2_LI)]
emissions_factor_df2_NIMH = df1_emissions_subset_t[df1_emissions_subset_t['alph_index'].isin(codes_df2_NIMH)]
emissions_factor_df3 = df1_emissions_subset_t[df1_emissions_subset_t['alph_index'].isin(codes_df3)]

In [724]:
## now select out material weightings from weighting dataframes that correspond to values in emissions table
emissions_factor_df3_merged = emissions_factor_df3.merge(df3_subset, left_on = 'alph_index', right_on = 'alph_index')
emissions_factor_df2_NIMH_merged = emissions_factor_df2_NIMH.merge(df2_subset_NIMH, left_on = 'alph_index', right_on = 'alph_index')
emissions_factor_df2_LI_merged = emissions_factor_df2_LI.merge(df2_subset_LI, left_on = 'alph_index', right_on = 'alph_index')

In [726]:
emissions_factor_df2_LI_merged

,alph_index,Material_x,VOC,CO,NOx,PM10,PM2.5,SOx,BC,OC,...,GHGs,Material_y,HEV: Conventional Material,HEV: Lightweight Material,PHEV: Conventional Material,PHEV: Lightweight Material,EV: Conventional Material,EV: Lightweight Material,FCV: Conventional Material,FCV: Lightweight Material
0,d,Average Steel,1.182879,8.551237,1.112527,0.650929,0.314066,4.186702,0.003714,0.010861,...,1111.82878,Steel,0.3145,0.3145,0.3444,0.3444,0.243,0.243,0.3057,0.3057
1,f,Stainless Steel,0.191237,2.616584,0.36596,0.504598,0.256584,0.474136,0.001828,0.006648,...,408.807658,Stainless Steel,0.0413,0.0413,0.0534,0.0534,0.0444,0.0444,0.0407,0.0407
2,j,Average Wrought Aluminum,1.116911,2.89177,6.358664,1.459809,1.504802,18.232098,0.029301,0.075167,...,7139.863278,Wrought Aluminum,0.1718,0.1718,0.1439,0.1439,0.1214,0.1214,0.1709,0.1709
3,n,Average Cast Aluminum,0.19018,0.632449,1.278121,0.421311,0.254995,3.145596,0.006479,0.017741,...,1462.123863,Cast Aluminum,0,0,0,0,0,0,0,0
4,ac,Copper,0.47308,1.361152,2.023006,0.366274,0.2426,2.139476,0.01065,0.033757,...,2036.676021,Copper,0.1386,0.1386,0.0867,0.0867,0.0744,0.0744,0.1425,0.1425
5,ak,High-Density Polyethylene,0.405266,0.770676,1.099996,0.08404,0.062062,0.533759,0.005532,0.019543,...,959.001831,Plastic: Polyethylene,0,0,0,0,0,0,0,0
6,al,Polyethylene Terephthalate,0.512242,0.763857,1.125293,0.089464,0.066676,0.451999,0.008022,0.021884,...,1103.355156,Plastic: Polyethylene Terephthalate,0.0025,0.0025,0.0026,0.0026,0.0025,0.0025,0.0025,0.0025
7,am,Polypropylene,0.261058,0.549306,0.791376,0.072189,0.055758,0.352037,0.005528,0.014116,...,781.254279,Plastic: Polypropylene,0.0089,0.0089,0.0056,0.0056,0.0076,0.0076,0.0093,0.0093
8,an,Average Plastic,0.461642,2.46364,1.992767,0.506137,0.138633,4.326873,0.012519,0.026923,...,1638.744788,Plastic: Polymer,0.0074,0.0074,0.0152,0.0152,0.008,0.008,0.0069,0.0069
9,ba,Thermal Insulation,0.127361,0.490551,1.853026,0.698146,0.365521,0.715841,0.00603,0.016359,...,844.660709,Thermal Insulation,0.005,0.005,0.0052,0.0052,0.0038,0.0038,0.0049,0.0049


In [635]:
emissions_factor_df2_LI_merged = emissions_factor_df2_LI.merge(df2_subset_LI, on = 'alph_index', how = 'inner')
emissions_factor_df2_LI_merged = pd.concat([df2_subset_LI,  emissions_factor_df2_LI], axis = 1)
emissions_factor_df2_LI_merged = pd.merge(emissions_factor_df2_LI, df2_subset_LI, on=['alph_index','alph_index'])
emissions_factor_df2_LI_merged.alph_index = emissions_factor_df2_LI_merged.alph_index.str.strip()
df2_subset_LI.alph_index = df2_subset_LI.alph_index.str.strip()
# emissions_factor_df2_LI_merged = pd.merge(emissions_factor_df2_LI, df2_subset_LI, 
#                                           left_on='alph_index', right_index=True)
emissions_factor_df2_LI_merged = pd.merge(emissions_factor_df2_LI.assign(alph_index=emissions_factor_df2_LI.alph_index.astype(str)), 
         df2_subset_LI.assign(alph_index=df2_subset_LI.alph_index.astype(str)), 
         how='left', on='alph_index')
left, right = emissions_factor_df2_LI.align(other, join="outer", axis=1)



In [653]:
emissions_factor_df3_merged.head(50)

,alph_index,Material_x,VOC,CO,NOx,PM10,PM2.5,SOx,BC,OC,...,ICEV: Conventional Material,ICEV: Lightweight Material,HEV: Conventional Material,HEV: Lightweight Material,PHEV: Conventional Material,PHEV: Lightweight Material,EV: Conventional Material,EV: Lightweight Material,FCV: Conventional Material,FCV: Lightweight Material
0,d,Average Steel,1.182879,8.551237,1.112527,0.650929,0.314066,4.186702,0.003714,0.010861,...,0.599226,0.307047,0.594983,0.332273,0.592058,0.343071,0.596579,0.228142,0.538544,0.204889
1,f,Stainless Steel,0.191237,2.616584,0.36596,0.504598,0.256584,0.474136,0.001828,0.006648,...,0,0.010302,0.000329,0.008448,0.000389,0.008242,0.001043,0.000754,0.033475,0.027282
2,g,Final Iron Product Mix,0.845665,0.478949,0.632618,0.420749,0.192477,1.46132,0.002798,0.007378,...,0.018932,0.035177,0.054031,0.033368,0.05444,0.034478,0,0.028704,0.000063,0.024888
3,j,Average Wrought Aluminum,1.116911,2.89177,6.358664,1.459809,1.504802,18.232098,0.029301,0.075167,...,0.046439,0.067357,0.027011,0.058452,0.025635,0.048169,0.030309,0.067567,0.038075,0.06336
4,n,Average Cast Aluminum,0.19018,0.632449,1.278121,0.421311,0.254995,3.145596,0.006479,0.017741,...,0.082414,0.140735,0.098459,0.143377,0.103426,0.151672,0.107036,0.164813,0.077647,0.129755
5,u,Average Nickel,2.226911,5.531022,5.93667,0.66175,0.461239,13.888695,0.046597,0.094481,...,0,0,0.000033,0.000032,0.000039,0.000034,0.000122,0.000126,0.000073,0.000077
6,ac,Copper,0.47308,1.361152,2.023006,0.366274,0.2426,2.139476,0.01065,0.033757,...,0.023788,0.030903,0.038626,0.04777,0.043507,0.050019,0.0583,0.076083,0.033341,0.042891
7,ad,Zinc,0.14072,0.554089,0.916072,0.947564,0.48073,0.740132,0.004102,0.018654,...,0,0,0.000166,0.000164,0.000208,0.000193,0.000662,0.000733,0.000417,0.000444
8,ag,Average Magnesium,0.555956,2.287094,3.368154,0.307493,0.214567,1.488756,0.025606,0.071416,...,0.000256,0.0038,0.000217,0.003505,0.000206,0.003224,0.000244,0.003775,0.000236,0.004256
9,ai,Glass,0.111276,0.481207,1.013502,0.059646,0.045048,0.417884,0.005294,0.013614,...,0.023509,0.032508,0.019943,0.029205,0.018932,0.026153,0.0224,0.030204,0.021706,0.034046


In [640]:
emissions_factor_df2_LI_merged = emissions_factor_df2_LI.merge(df2_subset_LI, left_on = 'alph_index', right_on = 'alph_index', how = 'outer')


In [646]:
test_df_1 = pd.DataFrame()
test_df_1['index'] = ['a', 'b', 'c', 'd', 'e']
test_df_1['vals'] = [1,2,3,4,5]
test_df_2 = pd.DataFrame()
test_df_2['index'] = ['b', 'c', 'a', 'e', 'd']
test_df_2['vals'] = [1,2,3,4,5]

In [647]:
test_merge = test_df_1.merge(test_df_2, left_on = 'index', right_on = 'index')

In [648]:
test_merge

,index,vals_x,vals_y
0,a,1,3
1,b,2,1
2,c,3,2
3,d,4,5
4,e,5,4


In [649]:
df2_subset_LI

48,alph_index,Material,HEV: Conventional Material,HEV: Lightweight Material,PHEV: Conventional Material,PHEV: Lightweight Material,EV: Conventional Material,EV: Lightweight Material,FCV: Conventional Material,FCV: Lightweight Material
68,bi,Active Material,0.1029,0.1029,0.156,0.156,0.2504,0.2504,0.1079,0.1079
69,bt,Graphite/Carbon,0.0627,0.0627,0.0915,0.0915,0.1421,0.1421,0.0655,0.0655
70,bu,Silicon,0,0,0,0,0,0,0,0
71,cc,Binder,0.0034,0.0034,0.005,0.005,0.008,0.008,0.0035,0.0035
72,bv,Copper,0.1386,0.1386,0.0867,0.0867,0.0744,0.0744,0.1425,0.1425
73,ac,Wrought Aluminum,0.1718,0.1718,0.1439,0.1439,0.1214,0.1214,0.1709,0.1709
74,j,Cast Aluminum,0,0,0,0,0,0,0,0
75,n,Electrolyte: LiPF6,0.007,0.007,0.0059,0.0059,0.0093,0.0093,0.0073,0.0073
76,bw,Electrolyte: Ethylene Carbonate,0.0196,0.0196,0.0166,0.0166,0.0261,0.0261,0.0205,0.0205
77,bx,Electrolyte: Dimethyl Carbonate,0.0196,0.0196,0.0166,0.0166,0.0261,0.0261,0.0205,0.0205


In [644]:
emissions_factor_df2_LI_merged

,alph_index,Material_x,VOC,CO,NOx,PM10,PM2.5,SOx,BC,OC,...,GHGs,Material_y,HEV: Conventional Material,HEV: Lightweight Material,PHEV: Conventional Material,PHEV: Lightweight Material,EV: Conventional Material,EV: Lightweight Material,FCV: Conventional Material,FCV: Lightweight Material
0,d,Average Steel,1.182879,8.551237,1.112527,0.650929,0.314066,4.186702,0.003714,0.010861,...,1111.82878,Coolant: Glycol,0.0366,0.0366,0.0298,0.0298,0.0242,0.0242,0.036,0.036
1,f,Stainless Steel,0.191237,2.616584,0.36596,0.504598,0.256584,0.474136,0.001828,0.006648,...,408.807658,Electronic Parts,0.0568,0.0568,0.0215,0.0215,0.0086,0.0086,0.0539,0.0539
2,j,Average Wrought Aluminum,1.116911,2.89177,6.358664,1.459809,1.504802,18.232098,0.029301,0.075167,...,7139.863278,Cast Aluminum,0,0,0,0,0,0,0,0
3,n,Average Cast Aluminum,0.19018,0.632449,1.278121,0.421311,0.254995,3.145596,0.006479,0.017741,...,1462.123863,Electrolyte: LiPF6,0.007,0.007,0.0059,0.0059,0.0093,0.0093,0.0073,0.0073
4,ac,Copper,0.47308,1.361152,2.023006,0.366274,0.2426,2.139476,0.01065,0.033757,...,2036.676021,Wrought Aluminum,0.1718,0.1718,0.1439,0.1439,0.1214,0.1214,0.1709,0.1709
5,ak,High-Density Polyethylene,0.405266,0.770676,1.099996,0.08404,0.062062,0.533759,0.005532,0.019543,...,959.001831,Plastic: Polyethylene Terephthalate,0.0025,0.0025,0.0026,0.0026,0.0025,0.0025,0.0025,0.0025
6,al,Polyethylene Terephthalate,0.512242,0.763857,1.125293,0.089464,0.066676,0.451999,0.008022,0.021884,...,1103.355156,Thermal Insulation,0.005,0.005,0.0052,0.0052,0.0038,0.0038,0.0049,0.0049
7,am,Polypropylene,0.261058,0.549306,0.791376,0.072189,0.055758,0.352037,0.005528,0.014116,...,781.254279,Plastic: Polyethylene,0,0,0,0,0,0,0,0
8,an,Average Plastic,0.461642,2.46364,1.992767,0.506137,0.138633,4.326873,0.012519,0.026923,...,1638.744788,Steel,0.3145,0.3145,0.3444,0.3444,0.243,0.243,0.3057,0.3057
9,ba,Thermal Insulation,0.127361,0.490551,1.853026,0.698146,0.365521,0.715841,0.00603,0.016359,...,844.660709,Stainless Steel,0.0413,0.0413,0.0534,0.0534,0.0444,0.0444,0.0407,0.0407


In [650]:
codes_df2_LI

['bi',
 'bt',
 'bu',
 'cc',
 'bv',
 'ac',
 'j',
 'n',
 'bw',
 'bx',
 'by',
 'am',
 'ak',
 'an',
 'al',
 'd',
 'f',
 'ch',
 'ba',
 'ca',
 'bz']

In [628]:
type(df2_subset_LI['alph_index'].iloc[0])

str

In [651]:
df2_subset_LI

48,alph_index,Material,HEV: Conventional Material,HEV: Lightweight Material,PHEV: Conventional Material,PHEV: Lightweight Material,EV: Conventional Material,EV: Lightweight Material,FCV: Conventional Material,FCV: Lightweight Material
68,bi,Active Material,0.1029,0.1029,0.156,0.156,0.2504,0.2504,0.1079,0.1079
69,bt,Graphite/Carbon,0.0627,0.0627,0.0915,0.0915,0.1421,0.1421,0.0655,0.0655
70,bu,Silicon,0,0,0,0,0,0,0,0
71,cc,Binder,0.0034,0.0034,0.005,0.005,0.008,0.008,0.0035,0.0035
72,bv,Copper,0.1386,0.1386,0.0867,0.0867,0.0744,0.0744,0.1425,0.1425
73,ac,Wrought Aluminum,0.1718,0.1718,0.1439,0.1439,0.1214,0.1214,0.1709,0.1709
74,j,Cast Aluminum,0,0,0,0,0,0,0,0
75,n,Electrolyte: LiPF6,0.007,0.007,0.0059,0.0059,0.0093,0.0093,0.0073,0.0073
76,bw,Electrolyte: Ethylene Carbonate,0.0196,0.0196,0.0166,0.0166,0.0261,0.0261,0.0205,0.0205
77,bx,Electrolyte: Dimethyl Carbonate,0.0196,0.0196,0.0166,0.0166,0.0261,0.0261,0.0205,0.0205


In [558]:
emissions_df2_LI_merged = emissions_factor_df2_LI.merge(df2_subset_LI, left_on='alph_index', right_on='alph_index')

In [636]:
emissions_factor_df2_LI_merged

,alph_index,Material_x,VOC,CO,NOx,PM10,PM2.5,SOx,BC,OC,...,GHGs,Material_y,HEV: Conventional Material,HEV: Lightweight Material,PHEV: Conventional Material,PHEV: Lightweight Material,EV: Conventional Material,EV: Lightweight Material,FCV: Conventional Material,FCV: Lightweight Material
0,d,Average Steel,1.182879,8.551237,1.112527,0.650929,0.314066,4.186702,0.003714,0.010861,...,1111.82878,Coolant: Glycol,0.0366,0.0366,0.0298,0.0298,0.0242,0.0242,0.036,0.036
1,f,Stainless Steel,0.191237,2.616584,0.36596,0.504598,0.256584,0.474136,0.001828,0.006648,...,408.807658,Electronic Parts,0.0568,0.0568,0.0215,0.0215,0.0086,0.0086,0.0539,0.0539
2,j,Average Wrought Aluminum,1.116911,2.89177,6.358664,1.459809,1.504802,18.232098,0.029301,0.075167,...,7139.863278,Cast Aluminum,0,0,0,0,0,0,0,0
3,n,Average Cast Aluminum,0.19018,0.632449,1.278121,0.421311,0.254995,3.145596,0.006479,0.017741,...,1462.123863,Electrolyte: LiPF6,0.007,0.007,0.0059,0.0059,0.0093,0.0093,0.0073,0.0073
4,ac,Copper,0.47308,1.361152,2.023006,0.366274,0.2426,2.139476,0.01065,0.033757,...,2036.676021,Wrought Aluminum,0.1718,0.1718,0.1439,0.1439,0.1214,0.1214,0.1709,0.1709
5,ak,High-Density Polyethylene,0.405266,0.770676,1.099996,0.08404,0.062062,0.533759,0.005532,0.019543,...,959.001831,Plastic: Polyethylene Terephthalate,0.0025,0.0025,0.0026,0.0026,0.0025,0.0025,0.0025,0.0025
6,al,Polyethylene Terephthalate,0.512242,0.763857,1.125293,0.089464,0.066676,0.451999,0.008022,0.021884,...,1103.355156,Thermal Insulation,0.005,0.005,0.0052,0.0052,0.0038,0.0038,0.0049,0.0049
7,am,Polypropylene,0.261058,0.549306,0.791376,0.072189,0.055758,0.352037,0.005528,0.014116,...,781.254279,Plastic: Polyethylene,0,0,0,0,0,0,0,0
8,an,Average Plastic,0.461642,2.46364,1.992767,0.506137,0.138633,4.326873,0.012519,0.026923,...,1638.744788,Steel,0.3145,0.3145,0.3444,0.3444,0.243,0.243,0.3057,0.3057
9,ba,Thermal Insulation,0.127361,0.490551,1.853026,0.698146,0.365521,0.715841,0.00603,0.016359,...,844.660709,Stainless Steel,0.0413,0.0413,0.0534,0.0534,0.0444,0.0444,0.0407,0.0407


In [262]:
for i in df1_emissions_subset_t['Material']:
    print(i)

Virgin Steel
Recycled Steel
Average Steel
Steel Credit
Stainless Steel
Final Iron Product Mix
Virgin Wrought Aluminum
Recycled Wrought Aluminum
Average Wrought Aluminum
Wrought Aluminum Credit
Virgin Cast Aluminum
Recycled Cast Aluminum
Average Cast Aluminum
Cast Aluminum Credit
Virgin Lead
Recycled Lead
Average Lead
Virgin Nickel
Recycled Nickel
Average Nickel
Virgin Nickel Hydroxide
Recycled Nickel Hydroxide
Average Nickel Hydroxide
Refined Nickel
Virgin Cobalt Oxide
Virgin Cobalt Chloride
Virgin Cobalt Metal
Copper
Zinc
Virgin Magnesium
Recycled Magnesium
Average Magnesium
Sulfuric Acid
Glass
Fiber Glass
High-Density Polyethylene
Polyethylene Terephthalate
Polypropylene
Average Plastic
Glass Fiber Composite Plastic
Carbon Fiber Composite Plastic for General Use
Carbon Fiber Composite Plastic for High Pressure Vessels
Rubber
Platinum
Vanadium
Zirconium
Titanium
Chromium
Rare Earth (Ni-MH)
Final Manganese
Manganese
Thermal Insulation
Carbon Fiber
Carbon Paper
Nafion117 Sheet
Nafion Dr

In [261]:

for i in df3_subset['Material']:
    print(i)

Stainless Steel
Cast Iron
Wrought Aluminum
Cast Aluminum
Copper/Brass
Zinc
Magnesium
Glass
Average Plastic
Rubber
Carbon Fiber-Reinforced Plastic for General Use
Carbon Fiber-Reinforced Plastic for High Pressure Vessels
Glass Fiber-Reinforced Plastic
Nickel
PFSA
Carbon Paper
PTFE
Carbon & PFSA Suspension
Platinum
Silicon
Carbon
Nd(Dy)FeB magnet
Phenolic resin
Enamel
PET
PBT
Mica
Glass fiber
Silicone
Epoxy resin
Nylon
Methacrylate ester resin
Paint/Varnish
Alumina
Gold
Polypropylene
Polyurethane
Zinc oxide
Others


In [263]:
len(df1_df3_merge)
Polypropylene
Polypropylene

16

In [264]:
for i in df1_df3_merge['Material']:
    print(i)

Stainless Steel
Zinc
Glass
Polypropylene
Average Plastic
Rubber
Platinum
Carbon Paper
PTFE
Silicon
Epoxy resin
Nd(Dy)FeB magnet
Alumina
Gold
Zinc oxide
Polyurethane


In [247]:
len(df3)

601

In [126]:
## clean and index epa curb weight data
epa_test_data_grouped = epa_test_data[epa_test_data['Model Year'] == 2023]
epa_test_data_grouped = epa_test_data_grouped.groupby(['Represented Test Vehicle Make', 'Represented Test Vehicle Model'], group_keys = False)
epa_curb_weights = epa_test_data_grouped.mean()
epa_curb_weights = epa_curb_weights.reset_index() ## flatten
epa_curb_weights = epa_curb_weights.reset_index() ## create unique id
epa_curb_weights = epa_curb_weights.rename(columns = {'index':'VehicleID'})

## clean and index epa adj weight data
test_data_2023 = test_data_2023[test_data_2023['Model Year'] == 2023]
test_data_2023_grouped = test_data_2023.groupby(['Represented Test Veh Make', 'Represented Test Veh Model', 'Veh Mfr Code'], group_keys = False).mean()
test_data_2023_grouped = test_data_2023_grouped.reset_index()
test_data_2023_grouped = test_data_2023_grouped.reset_index()
test_data_2023_grouped = test_data_2023_grouped.rename(columns = {'index':'VehicleID'})


# select 2023 years from epa range/efficiency data
epa_df = epa_df[epa_df['year'] == 2023]

## rename columns of each df
rename_dict = {'id': 'VehicleID', 'make': 'Make', 'model':'Model', 'mfrCode':'MFRCode', 'range': 'Range',
              'rangeHwy': 'RangeHwy', 'year':'Year'}
epa_df = epa_df.rename(rename_dict, axis=1)  
rename_dict = {'Model Year': 'Year', 'Represented Test Vehicle Model':'Model', 
               'Represented Test Vehicle Make': 'Make', 'Curb Weight (lbs.)':'CurbWeight'}
epa_curb_weights = epa_curb_weights.rename(rename_dict, axis=1)  
rename_dict = {'Model Year': 'Year', 'Represented Test Veh Make':'Make', 'Veh Mfr Code': 'MFRCode',
               'Represented Test Veh Model':'Model', 'Equivalent Test Weight (lbs.)':'AdjWeight'}
epa_adj_weights = test_data_2023_grouped.rename(rename_dict, axis=1)  

## take selected columns from each df
epa_df = epa_df[['VehicleID', 'Year', 'MFRCode', 'Make', 'Model', 'Range', 'RangeHwy', 'UHighway', 'UCity']]
epa_curb_weights = epa_curb_weights[['VehicleID', 'Year', 'Make', 'Model', 'CurbWeight']]
epa_adj_weights = epa_adj_weights[['VehicleID', 'Year', 'MFRCode', 'Make', 'Model', 'AdjWeight']]

## convert years back to int
epa_curb_weights.astype({'Year': 'int32'})
epa_adj_weights.astype({'Year': 'int32'})


<ipython-input-126-0dd80e7b8919>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  epa_curb_weights = epa_test_data_grouped.mean()
<ipython-input-126-0dd80e7b8919>:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_data_2023_grouped = test_data_2023.groupby(['Represented Test Veh Make', 'Represented Test Veh Model', 'Veh Mfr Code'], group_keys = False).mean()


,VehicleID,Year,MFRCode,Make,Model,AdjWeight
0,0,2023,HNX,ACURA,INTEGRA,3375.0
1,1,2023,HNX,ACURA,INTEGRA A-SPEC,3437.5
2,2,2023,HNX,ACURA,MDX AWD,4750.0
3,3,2023,HNX,ACURA,MDX AWD Type-S,5000.0
4,4,2023,HNX,ACURA,MDX FWD,4500.0
...,...,...,...,...,...,...
632,632,2023,VVX,Volvo,XC60 B6 AWD,4750.0
633,633,2023,VVX,Volvo,XC60 T8 AWD Recharge Extended Range,5000.0
634,634,2023,VVX,Volvo,XC90 B5 AWD,5000.0
635,635,2023,VVX,Volvo,XC90 B6 AWD,5000.0


In [127]:
## clean and index epa adj weight data
test_data_2023 = test_data_2023[test_data_2023['Model Year'] == 2023]
test_data_2023_grouped = test_data_2023.groupby(['Represented Test Veh Make', 'Represented Test Veh Model'], group_keys = False).mean()
test_data_2023_grouped = test_data_2023_grouped.reset_index()
test_data_2023_grouped = test_data_2023_grouped.rename(columns = {'index':'VehicleID'})


<ipython-input-127-fd38c6ab2a38>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_data_2023_grouped = test_data_2023.groupby(['Represented Test Veh Make', 'Represented Test Veh Model'], group_keys = False).mean()


In [195]:
epa_df['VehicleID'].is_unique

True

In [183]:
epa_curb_weights.head()

,VehicleID,Year,Make,Model,CurbWeight
0,0,2023.0,2022,Lucid Air Dream P,5203.0
1,1,2023.0,2022,Lucid Air Grand Touring,5203.0
2,2,2023.0,2023,Lucid Air Touring,4987.0
3,3,2023.0,ACURA,INTEGRA A-SPEC,3146.5
4,4,2023.0,ACURA,MDX AWD,4565.0


In [184]:
epa_adj_weights.head()

,VehicleID,Year,MFRCode,Make,Model,AdjWeight
0,0,2023.0,HNX,ACURA,INTEGRA,3375.0
1,1,2023.0,HNX,ACURA,INTEGRA A-SPEC,3437.5
2,2,2023.0,HNX,ACURA,MDX AWD,4750.0
3,3,2023.0,HNX,ACURA,MDX AWD Type-S,5000.0
4,4,2023.0,HNX,ACURA,MDX FWD,4500.0


In [185]:
# ## clean zip data (drop unused columns)
# if len(zip_to_grid.columns) > 6:
#     zip_to_grid = zip_to_grid.drop(zip_to_grid.columns[[0,6,7]], axis = 1)
# ## clean epa data (drop unused columns)
# epa_df = epa_df[['id', 'make', 'model', 'mfrCode', 'range', 'rangeHwy', 'UHighway', 'UCity', 'year']]
# ## epa_df from 2023 
# epa_df = epa_df[epa_df['year'] == 2023]
# ## rename columns 
# rename_dict = {'id': 'VehicleID', 'make': 'Make', 'model':'Model', 'mfrCode':'MFRCode', 'range': 'Range',
#               'rangeHwy': 'RangeHwy', 'year':'Year'}
# epa_df = epa_df.rename(rename_dict, axis=1)  
# ## clean epa test data
# epa_test_data = epa_test_data[['Vehicle ID', 'Model Year', 'Represented Test Vehicle Make', 'Represented Test Vehicle Model', 'Curb Weight (lbs.)']]
# epa_test_data = epa_test_data[epa_test_data['Model Year'] == 2023]
# ## rename columns
# rename_dict = {'Model Year': 'Year', 'Represented Test Vehicle Model':'Model', 
#                'Represented Test Vehicle Make': 'Make', 'Curb Weight (lbs.)':'CurbWeight'}
# epa_test_data = epa_test_data.rename(rename_dict, axis=1)  
# ##
# epa_weight_data = test_data_2023[['Test Vehicle ID', 'Model Year', 'Veh Mfr Code', 'Represented Test Veh Make', 'Represented Test Veh Model', 'Equivalent Test Weight (lbs.)']]
# epa_weight_data = epa_weight_data[epa_weight_data['Model Year'] == 2023]
# ## rename weight data
# rename_dict = {'Test Vehicle ID': 'VehicleID', 'Model Year': 'Year', 'Represented Test Veh Make':'Make', 
#                'Represented Test Veh Model':'Model', 'Veh Mfr Code':'MFRCode', 'Equivalent Test Weight (lbs.)':'AdjWeight'}
# epa_weight_data = epa_weight_data.rename(rename_dict, axis=1)  

In [196]:
# zip_to_grid.head()
## convert to sqlite db
db_conn = sql.connect("/Users/josheverts/EV_data/EV_data.db")

In [197]:
epa_df.VehicleID.is_unique

True

In [198]:
c = db_conn.cursor()

In [255]:
## range 
c.execute("""DROP TABLE IF EXISTS EPA_range_data;""")
c.execute(
    """
    CREATE TABLE IF NOT EXISTS EPA_range_data(
        VehicleID INTEGER,
        Year REAL,
        MFRCode TEXT NOT NULL,
        Make TEXT NOT NULL COLLATE NOCASE ,
        Model TEXT NOT NULL COLLATE NOCASE,
        Range REAL,
        RangeHwy REAL,
        UHighway REAL,
        UCity REAL,
        
        PRIMARY KEY(VehicleID),
        FOREIGN KEY(Make) REFERENCES EPA_curb_weight_data(Make),
        FOREIGN KEY(Model) REFERENCES EPA_curb_weight_data(Model),
        FOREIGN KEY(Make) REFERENCES EPA_adj_weight_data(Make),
        FOREIGN KEY(Model) REFERENCES EPA_adj_weight_data(Model),
        FOREIGN KEY(MFRCode) REFERENCES EPA_adj_weight_data(MFRCode)
        );
     """
)

# curb weight
c.execute("""DROP TABLE IF EXISTS EPA_curb_weight_data""")
c.execute(
    """
    CREATE TABLE IF NOT EXISTS EPA_curb_weight_data (
        VehicleID INTEGER,
        Year REAL,
        Make TEXT NOT NULL COLLATE NOCASE ,
        Model TEXT NOT NULL COLLATE NOCASE,
        CurbWeight Real,
        PRIMARY KEY(VehicleID)
        );
     """
)

# adj weight
c.execute("""DROP TABLE IF EXISTS EPA_adj_weight_data;""")
c.execute(
    """
    CREATE TABLE IF NOT EXISTS EPA_adj_weight_data (
        VehicleID INTEGER,
        Year REAL,
        MFRCode TEXT NOT NULL,
        Make TEXT NOT NULL COLLATE NOCASE ,
        Model TEXT NOT NULL COLLATE NOCASE,
        AdjWeight Real,
        PRIMARY KEY(VehicleID)
        );
    """
)


In [256]:
#zip_to_grid.to_sql('zip2grid', db_conn, if_exists='append', index=False)
epa_df.to_sql('EPA_range_data', db_conn, if_exists='append', index=False)
epa_curb_weights.to_sql('EPA_curb_weight_data', db_conn, if_exists='append', index=False)
epa_adj_weights.to_sql('EPA_adj_weight_data', db_conn, if_exists='append', index=False)
#zip_to_grid.to_sql('zip2grid', db_conn, if_exists='append', index=False)
#zip_to_grid.to_sql('zip2grid', db_conn, if_exists='append', index=False)

637

In [257]:
pd.read_sql("SELECT * FROM EPA_adj_weight_data LIMIT 5", db_conn)

,VehicleID,Year,MFRCode,Make,Model,AdjWeight
0,0,2023.0,HNX,ACURA,INTEGRA,3375.0
1,1,2023.0,HNX,ACURA,INTEGRA A-SPEC,3437.5
2,2,2023.0,HNX,ACURA,MDX AWD,4750.0
3,3,2023.0,HNX,ACURA,MDX AWD Type-S,5000.0
4,4,2023.0,HNX,ACURA,MDX FWD,4500.0


In [258]:
## some test joins
query = '''SELECT DISTINCT *
         FROM EPA_range_data 
         INNER JOIN EPA_adj_weight_data
         ON EPA_range_data.Make = EPA_adj_weight_data.Make AND 
         EPA_range_data.Model = EPA_adj_weight_data.Model'''

c.execute(query)
# Fetching rows from the result table
result = c.fetchall()
for row in result:
    print(row)

(45099, 2023.0, 'JLX', 'Land Rover', 'Defender 90', 0.0, 0.0, 30.5805, 22.3491, 354, 2023.0, 'JLX', 'Land Rover', 'Defender 90', 5500.0)
(45103, 2023.0, 'JLX', 'Land Rover', 'Discovery', 0.0, 0.0, 33.7329, 24.3528, 327, 2023.0, 'JLX', 'LAND ROVER', 'Discovery', 5500.0)
(45104, 2023.0, 'JLX', 'Land Rover', 'Discovery MHEV', 0.0, 0.0, 35.5392, 24.4829, 355, 2023.0, 'JLX', 'Land Rover', 'Discovery MHEV', 5500.0)
(45105, 2023.0, 'JLX', 'Land Rover', 'Defender 110', 0.0, 0.0, 29.4554, 20.6902, 352, 2023.0, 'JLX', 'Land Rover', 'Defender 110', 6000.0)
(45108, 2023.0, 'JLX', 'Land Rover', 'Defender 90', 0.0, 0.0, 26.2277, 18.3475, 354, 2023.0, 'JLX', 'Land Rover', 'Defender 90', 5500.0)
(45109, 2023.0, 'JLX', 'Land Rover', 'Defender 110', 0.0, 0.0, 25.5562, 17.6252, 352, 2023.0, 'JLX', 'Land Rover', 'Defender 110', 6000.0)
(45158, 2023.0, 'JLX', 'Land Rover', 'Discovery Sport', 0.0, 0.0, 34.2146, 23.2538, 356, 2023.0, 'JLX', 'Land Rover', 'Discovery Sport', 4750.0)
(45189, 2023.0, 'JLX', 'Lan

In [82]:
## zip2grid database
zip_to_grid = pd.read_excel('/Users/josheverts/Downloads/Zip_to_Grid.xlsx')
## rename columns 
rename_dict = {'ZIP (numeric)': 'ZIP', 'state': 'State', 'eGRID Subregion #1': 'Subregion1', 'eGRID Subregion #2': 'Subregion2',
              'eGRID Subregion #3': 'Subregion3',}
zip_to_grid = zip_to_grid.rename(rename_dict, axis=1)  
zip_to_grid = zip_to_grid[['ZIP', 'State', 'Subregion1', 'Subregion2', 'Subregion3']]
len(zip_to_grid)

42176

In [83]:
zip_to_grid = zip_to_grid.drop_duplicates(['ZIP'])

In [84]:
len(zip_to_grid)

41526

In [24]:
## subregion emissions database
subregion_emissions = pd.read_excel('/Users/josheverts/Downloads/power_profiler_zipcode_tool.xlsx', 
                                    sheet_name = 'eGRID Subregion Emission Factor', header = 2)
## drop first row (set of codes)
subregion_emissions = subregion_emissions.drop(index = 0)
rename_dict = {'eGRID subregion name ': 'RegionName', 'eGRID subregion acronym': 'RegionCode',
               'eGRID subregion annual CO2 total output emission rate (lb/MWh)': 'Co2Rate'}
subregion_emissions = subregion_emissions.rename(rename_dict, axis=1)  
subregion_emissions = subregion_emissions[['RegionName', 'RegionCode', 'Co2Rate']]

In [56]:
## create gridmix database 
db_conn = sql.connect("/Users/josheverts/EV_data/gridmix.db")
c = db_conn.cursor()


In [57]:
c.execute("""DROP TABLE IF EXISTS Zip2Grid;""")

c.execute(
    """
    CREATE TABLE IF NOT EXISTS Zip2Grid (
        ZIP INTEGER,
        State TEXT NOT NULL,
        Subregion1 TEXT NOT NULL,
        Subregion2 TEXT,
        Subregion3 TEXT,
        Co2Rate REAL,
        
        PRIMARY KEY(ZIP),
        FOREIGN KEY(Subregion1) REFERENCES Subregion(RegionCode),
        FOREIGN KEY(Subregion2) REFERENCES Subregion(RegionCode),
        FOREIGN KEY(Subregion3) REFERENCES Subregion(RegionCode)
        );
     """
)
# curb weight
c.execute("""DROP TABLE IF EXISTS Subregion;""")
c.execute(
    """
    CREATE TABLE IF NOT EXISTS Subregion (
        RegionName TEXT NOT NULL,
        RegionCode TEXT NOT NULL,
        Co2Rate REAL,
        PRIMARY KEY(RegionCode)
        );
     """
)



In [85]:
zip_to_grid.to_sql('Zip2Grid', db_conn, if_exists='append', index=False)

41526

In [86]:
subregion_emissions.to_sql('Subregion', db_conn, if_exists='append', index=False)

27